In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
import math

In [3]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [14]:
#普通神经元weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
#普通神经元bias
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
def LSTMLayer(input,unitNum,outputDimen,keep_prob):
    #input shape is [batchsize,vector的最大长度,数据个数]
    weight= weight_variable([unitNum,outputDimen])
    bians = bias_variable([outputDimen]) 
    lstn_cell = tf.contrib.rnn.BasicLSTMCell(unitNum)
    #final_state[0] 维度=[batch_size,unitNum] is 最终存的Cell值
    #final_state[1] 维度=[batch_size,unitNum] is 最终的输出值
    #outputs 维度=[batch_size,time_steps,unitNum]如果time_major=False
    #outputs 维度=[time_steps,batch_size,unitNum]如果time_major=True
    #outputs记录了每一个批次的数据的每一次过Cell的最终输出值
    outputs,final_state = tf.nn.dynamic_rnn(lstn_cell,input,dtype=tf.float32)
    result = tf.matmul(final_state[1],weight) + bians
    output = tf.nn.dropout(result,keep_prob)
    return output

In [15]:
tf.reset_default_graph()
n_inputs = 28 #一行28个数据
time_steps = 28 #一共28行
batch_size = 1024
n_batch = math.ceil(mnist.train.num_examples / batch_size)
max_steps = 30

with tf.name_scope("Input"):
    x = tf.placeholder(tf.float32,[None,784],name='x') #28*28=784
    y = tf.placeholder(tf.float32,[None,10],name='y') #28*28=784
keep_prob = tf.placeholder(tf.float32)


reshape_x = tf.reshape(x,[-1,time_steps,n_inputs])
#LSTM层 100个
with tf.name_scope("LSTM_layer1"):
    lstmResult = LSTMLayer(reshape_x,100,10,keep_prob)

#loss
with tf.name_scope("Loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=lstmResult))
    tf.summary.scalar("Loss",loss)

#train
with tf.name_scope("Train"):
    train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

#accuracy
with tf.name_scope("Accuracy"):
    predictionIndex = tf.argmax(tf.nn.softmax(lstmResult),1)
    realIndex = tf.argmax(y,1)
    correctBools = tf.equal(predictionIndex,realIndex)
    accuracy = tf.reduce_mean(tf.cast(correctBools,tf.float32))
    tf.summary.scalar("Accuracy",accuracy)
        

summaryMerge = tf.summary.merge_all()

In [16]:
trainlog_path = "./logs\\train\\"
testlog_path= "./logs\\test\\"

#用来保存网络模型
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    with tf.summary.FileWriter(trainlog_path,sess.graph) as trainWriter:
        with tf.summary.FileWriter(testlog_path,sess.graph) as testWriter:
            startClock = time.perf_counter()
            print("Start Train.")
            for epoch in range(max_steps):
                startEpochTime = time.perf_counter()
                for batch in range(n_batch):
                    batch_xs,batch_ys = mnist.train.next_batch(batch_size)
                    trainResult,summary = sess.run([train_step,summaryMerge],feed_dict={x:batch_xs,y:batch_ys,keep_prob:1})
                    trainWriter.add_summary(summary,epoch)
                    batch_xs,batch_ys = mnist.test.next_batch(batch_size)
                    summary = sess.run(summaryMerge,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1})
                    testWriter.add_summary(summary,epoch)
                acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1})
                print("Iter = {0}, Testing Accuracy = {1},  TrainTime = {2}".format(epoch,acc,time.perf_counter() - startEpochTime))
            print("End Train.   Use Time = ",time.perf_counter() - startClock)
        #saver.save(sess, path_for_savedata, global_step=max_steps)

Start Train.
Iter = 0, Testing Accuracy = 0.321399986743927,  TrainTime = 2.8953825039998264
Iter = 1, Testing Accuracy = 0.3684999942779541,  TrainTime = 2.5928150509998886
Iter = 2, Testing Accuracy = 0.5027999877929688,  TrainTime = 2.559751378999863
Iter = 3, Testing Accuracy = 0.6407999992370605,  TrainTime = 2.521129020000444
Iter = 4, Testing Accuracy = 0.7325999736785889,  TrainTime = 2.574146577000647
Iter = 5, Testing Accuracy = 0.7856000065803528,  TrainTime = 2.5101997399997344
Iter = 6, Testing Accuracy = 0.821399986743927,  TrainTime = 2.5867572850002034
Iter = 7, Testing Accuracy = 0.8424000144004822,  TrainTime = 2.581100643999889
Iter = 8, Testing Accuracy = 0.8592000007629395,  TrainTime = 2.5293216230002145
Iter = 9, Testing Accuracy = 0.8715000152587891,  TrainTime = 2.505403344000115
Iter = 10, Testing Accuracy = 0.8806999921798706,  TrainTime = 2.5289304130001256
Iter = 11, Testing Accuracy = 0.8894000053405762,  TrainTime = 2.4878997900004833
Iter = 12, Testing A